In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from src.models import CrossEntropyClassification
from src.data import train_val_test_split, get_descriptor_and_labels
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, RichProgressBar
import torch

In [3]:
desc_type = "steinhardt"
use_mda = True
numb_train_samples = 8_000

In [4]:
train_structs, val_structs, test_structs = train_val_test_split(mda=use_mda,num_files=None)

In [5]:
len(train_structs), len(val_structs), len(test_structs)

(1285, 20, 1245)

In [6]:
train_x, train_y, label_mapping = get_descriptor_and_labels(train_structs, num_samples_per_type=numb_train_samples)
val_x, val_y, _ = get_descriptor_and_labels(val_structs, num_samples_per_type=2_500)
test_x, test_y, _ = get_descriptor_and_labels(test_structs, num_samples_per_type=2_500)

In [7]:
label_mapping

{'hda': 0, 'lda': 1, 'mda': 2}

In [8]:
from sklearn import preprocessing

# fit to training data
scaler = preprocessing.StandardScaler().fit(train_x)
scaled_train_x = torch.FloatTensor(scaler.transform(train_x))
scaled_val_x = torch.FloatTensor(scaler.transform(val_x))

In [9]:
train_dataset = TensorDataset(scaled_train_x,train_y)
val_dataset = TensorDataset(scaled_val_x,val_y)

train_loader = DataLoader(train_dataset, batch_size=250, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10000, shuffle=False)

output_size = train_y.shape[1]

In [10]:
import optuna
from src.data import predict_test_set_classes
from sklearn.metrics import balanced_accuracy_score
from pytorch_lightning.loggers import TensorBoardLogger

def optimise_NN(trial: optuna.Trial):
    # Optuna optimisation function for the NN
    
    # 1. Suggest the hyperparameters
    n_layers = trial.suggest_int("n_layers", 1, 5)
    neurons_per_layer = trial.suggest_int("n_units_l0", 8, 128, log=True)
    hidden_units = [neurons_per_layer] * n_layers
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    
    input_size = 30

    # 2. Create the model
    model = CrossEntropyClassification(
        input_size,
        *hidden_units,
        output_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
    )

    # 3. Train the model
    trainer = Trainer(
        accelerator="auto",
        max_epochs=200,
        callbacks=[
            RichProgressBar(),
            EarlyStopping(monitor="validation_loss", patience=10),
        ],
        logger=TensorBoardLogger("lightning_logs"),
    )
    trainer.fit(model, train_loader, val_loader)
    
    # 4. Load the best model
    model.load_state_dict(torch.load(trainer.checkpoint_callback.best_model_path)['state_dict'])
    
    # 5. Evaluate the model
    pred_classes, val_classes, _ = predict_test_set_classes(val_structs,model=model, scaler=scaler)
    
    return balanced_accuracy_score(val_classes, pred_classes)

In [12]:
study_name = "optimise_NN"  # Unique identifier of the study.
storage_name = f"sqlite:///{study_name}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction="maximize",load_if_exists=True)
study.optimize(optimise_NN, n_trials=10)

[I 2023-11-30 14:22:34,884] Using an existing study with name 'optimise_NN' instead of creating a new one.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name       ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ activation │ ReLU       │      0 │
│ 1 │ layers     │ ModuleList │  2.1 K │
└───┴────────────┴────────────┴────────┘

Trainable params: 2.1 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.1 K                                                                                                
Total estimated model params size (MB): 0

Output()

/Users/zoefaurebeaulieu/miniconda3/envs/steinhardt/lib/python3.10/site-packages/pytorch_lightning/trainer/connector
s/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may 
be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus 
on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

/Users/zoefaurebeaulieu/miniconda3/envs/steinhardt/lib/python3.10/site-packages/pytorch_lightning/trainer/connector
s/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which 
may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of 
cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(